In [6]:
import psycopg2
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import plot 
import plotly.graph_objects as go
import warnings
import datetime
warnings.filterwarnings('ignore')

# import q_credentials.db_secmaster_cloud_cred as db_secmaster_cred
import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred


In [7]:
# connect to our securities_master database
conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)

In [8]:
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)

In [13]:
sql="""SELECT s.ticker, max(d.symbol_id), max(d.date_price)
FROM public.symbol s inner join d_data d on s.id= d.symbol_id
group by s.ticker"""
df_ticker_last_day=pd.read_sql(sql,con=conn_secmaster)

In [14]:
df_ticker_last_day

,ticker,max,max
0,CLF,5807,2020-11-06
1,RGA,9649,2020-11-06
2,RCS,9592,2020-11-06
3,NIM,8722,2020-11-06
4,CRVL,6007,2020-11-06
...,...,...,...
1159,BDN,5194,2020-11-06
1160,JBL,7806,2020-11-06
1161,LOW,8160,2020-11-06
1162,STKL,10211,2020-11-06


In [6]:
# sql="""select date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from daily_data where stock_id=71"""
# df_stock=pd.read_sql(sql,con=conn_secmaster)

In [7]:
# import datetime

In [15]:
sql="select * from indicator"
ind_list=list(pd.read_sql(sql,con=conn_indicator)['name'])
ind_list

['candle_1', 'candle_2', 'candle_3', 'anomaly', 'level', '', 'alpa_ind']

In [9]:
symbol_id='AAPL'#'BOM500114'
start_date=datetime.datetime(2018,1,1).strftime("%Y-%m-%d")
indicator_name = 'candle_1'

In [10]:
sql="select d.date_price as date, open_price as open, high_price as high, low_price as low , close_price as close,volume from daily_data d join symbol s on d.stock_id = s.id where s.ticker='%s' and d.date_price > '%s'" %(symbol_id, start_date)
df_price=pd.read_sql(sql,con=conn_secmaster)

In [11]:
sql="select d.date_price as date, d.value from daily_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, indicator_name, start_date)
df_indicator=pd.read_sql(sql,con=conn_indicator)

In [12]:
df_indicator.set_index('date',inplace=True)

In [13]:
df_indicator.head()

,value
date,
2018-01-31,"{'level': -10.0, 'pattern_name': 'short'}"
2018-02-01,"{'level': 0.0, 'pattern_name': ''}"
2018-02-02,"{'level': 10.0, 'pattern_name': 'long'}"
2018-02-05,"{'level': 0.0, 'pattern_name': ''}"
2018-02-06,"{'level': 10.0, 'pattern_name': 'long'}"


In [14]:
pd.options.display.max_columns = None

In [15]:
df_all_ind=pd.DataFrame()
for ind in ind_list:
    print(ind)
    sql="select d.date_price as date, d.value from daily_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s'" %(symbol_id, ind, start_date)
    df_indicator=pd.read_sql(sql,con=conn_indicator)
    if df_indicator.empty:
        continue:
    df_indicator.set_index('date',inplace=True)
    df_indicator=pd.concat([df_indicator.drop(['value'], axis=1), df_indicator['value'].apply(pd.Series)], axis=1)
    df_indicator.columns=[ind+"_"+col for col in df_indicator.columns]
    if df_all_ind.empty:
        df_all_ind=pd.merge(left=df_price, right=df_indicator,on='date')
    else:
        df_all_ind=pd.merge(left=df_all_ind, right=df_indicator,on='date')

candle_1
candle_2
candle_3
level
anomaly


In [16]:
df_all_ind.head()

,date,open,high,low,close,volume,candle_1_level,candle_1_pattern_name,candle_2_level,candle_2_pattern_name,candle_3_level,candle_3_pattern_name,level_level,level_nearest_support_atr_range,level_nearest_support_duration,level_nearest_support_range,level_support,level_nearest_resistance_atr_range,level_nearest_resistance_duration,level_nearest_resistance_range,level_resistance,anomaly_bear,anomaly_body_size,anomaly_body_size_dev,anomaly_body_size_ema,anomaly_bull,anomaly_gap,anomaly_gap_dev,anomaly_gap_ema,anomaly_gap_signal,anomaly_vol_anomaly,anomaly_vol_anomaly_2,anomaly_vol_dev,anomaly_vol_ema,anomaly_wick_bear,anomaly_wick_bear_dev,anomaly_wick_bear_ema,anomaly_wick_bear_signal,anomaly_wick_bull,anomaly_wick_bull_dev,anomaly_wick_bull_ema,anomaly_wick_bull_signal
0,2018-01-02,170.16,172.30,169.26,172.26,25555934,10.0,long,0.0,,0.0,,0.0,2.131653,61.0,"[165.28, 169.48]","[[165.28, 169.48, 2017-11-02 00:00:00, True], ...",0.672694,13.0,"[174.35, 173.25]","[[174.35, 173.25, 2017-12-20 00:00:00, False],...",0.0,2.10,1.267830,0.925995,1.0,0.93,0.008986,0.921717,0.008986,1.204958,0.0,0.049590,2.434848e+07,0.90,0.760427,0.511240,0.760427,0.04,-0.901577,0.406411,0.000000
1,2018-01-03,172.53,174.55,171.96,172.23,29495399,5.0,DOJI_SPINOFF,0.0,,0.0,,0.0,2.064320,62.0,"[165.28, 169.48]","[[165.28, 169.48, 2017-11-02 00:00:00, True], ...",0.668244,14.0,"[174.35, 173.25]","[[174.35, 173.25, 2017-12-20 00:00:00, False],...",1.0,0.30,0.630623,0.812178,0.0,0.27,-0.663854,0.803223,0.000000,0.525592,0.0,0.166551,2.528429e+07,0.27,-0.422309,0.467378,0.000000,2.02,1.886577,0.699791,1.886577
2,2018-01-04,172.54,173.47,172.08,173.03,22434597,0.0,,0.0,,0.0,,0.0,2.618706,63.0,"[165.28, 169.48]","[[165.28, 169.48, 2017-11-02 00:00:00, True], ...",0.356886,15.0,"[174.35, 173.25]","[[174.35, 173.25, 2017-12-20 00:00:00, False],...",0.0,0.49,0.349788,0.753600,1.0,0.31,-0.565550,0.713546,0.000000,0.316858,0.0,0.094143,2.476616e+07,0.46,-0.012953,0.466037,0.000000,0.44,-0.325729,0.652556,0.000000
3,2018-01-05,173.44,175.37,173.05,175.00,23660018,0.0,,0.0,,0.0,,0.0,3.473040,64.0,"[165.28, 169.48]","[[165.28, 169.48, 2017-11-02 00:00:00, True], ...",0.234726,15.0,"[176.02, 175.01]","[[176.02, 175.01, 2017-12-21 00:00:00, True], ...",0.0,1.56,0.732913,0.900218,1.0,0.41,-0.377237,0.658356,0.000000,0.705911,0.0,0.036842,2.456504e+07,0.39,-0.137572,0.452212,0.000000,0.37,-0.384546,0.601182,0.000000
4,2018-01-08,174.35,175.61,173.93,174.35,20567766,1.0,DOJI,-2.0,BEARISH HARAMI CROSS,0.0,,0.0,3.332960,67.0,"[165.28, 169.48]","[[165.28, 169.48, 2017-11-02 00:00:00, True], ...",0.557087,18.0,"[176.02, 175.01]","[[176.02, 175.01, 2017-12-21 00:00:00, True], ...",0.0,0.00,1.000000,0.736542,0.0,-0.65,-0.010408,0.656837,-0.000000,0.862805,0.0,0.137195,2.383827e+07,0.42,-0.059045,0.446355,0.000000,1.26,0.747652,0.720967,0.747652


In [17]:
df_level=df_all_ind[['date','high','low']]
df_level['res_outer']=df_all_ind['level_nearest_resistance_range'].str[0]
df_level['res_inner']=df_all_ind['level_nearest_resistance_range'].str[1]
df_level['sup_outer']=df_all_ind['level_nearest_support_range'].str[0]
df_level['sup_inner']=df_all_ind['level_nearest_support_range'].str[1]
df_level['target_resistance']=0
df_level['target_support']=0

In [18]:
df_level.head()

,date,high,low,res_outer,res_inner,sup_outer,sup_inner,target_resistance,target_support
0,2018-01-02,172.30,169.26,174.35,173.25,165.28,169.48,0,0
1,2018-01-03,174.55,171.96,174.35,173.25,165.28,169.48,0,0
2,2018-01-04,173.47,172.08,174.35,173.25,165.28,169.48,0,0
3,2018-01-05,175.37,173.05,176.02,175.01,165.28,169.48,0,0
4,2018-01-08,175.61,173.93,176.02,175.01,165.28,169.48,0,0


In [19]:
# Taking a forward look into the future and looking if the levels have been broken
for i, row in df_level.iterrows():
    for j in range(i+1,len(df_level)):
        if df_level.iloc[j]['high']>=df_level.iloc[i]['res_inner']:
            df_level.at[i,'target_resistance']=1
            break
        if df_level.iloc[j]['low']<=df_level.iloc[i]['sup_inner']:
            df_level.at[i,'target_support']=1
            break

In [20]:
len(df_level[df_level['target_resistance']==1])
len(df_level[df_level['target_support']==1])
len(df_level[(df_level['target_support']==0) & (df_level['target_resistance']==0)])

3

In [21]:
df_level.head()

,date,high,low,res_outer,res_inner,sup_outer,sup_inner,target_resistance,target_support
0,2018-01-02,172.30,169.26,174.35,173.25,165.28,169.48,1,0
1,2018-01-03,174.55,171.96,174.35,173.25,165.28,169.48,1,0
2,2018-01-04,173.47,172.08,174.35,173.25,165.28,169.48,1,0
3,2018-01-05,175.37,173.05,176.02,175.01,165.28,169.48,1,0
4,2018-01-08,175.61,173.93,176.02,175.01,165.28,169.48,1,0


In [22]:
df_feature_target=pd.merge(left=df_level[df_level.columns.difference(['low', 'high'])], right=df_all_ind,on='date')

In [23]:
columns=['date','target_resistance', 'target_support', 
'candle_1_pattern_name','candle_2_pattern_name', 'candle_3_pattern_name',
'level_nearest_support_atr_range','level_nearest_support_duration','level_nearest_resistance_atr_range','level_nearest_resistance_duration',
'anomaly_gap_signal', 'anomaly_vol_anomaly', 'anomaly_vol_anomaly_2','anomaly_wick_bear_signal','anomaly_wick_bull_signal']

In [5]:
df_feature_target[columns]

NameError: name 'df_feature_target' is not defined